# Libraries

In [1]:
import requests, bs4
import pandas as pd
from IPython.display import clear_output
import logging


pd.options.display.max_columns = 5
pd.options.display.max_colwidth = -1

# Scraping

In [2]:
days = []
days += [('january', str(i)) for i in range(1,32)]
days += [('february', str(i)) for i in range(1,30)]
days += [('march', str(i)) for i in range(1,32)]
days += [('april', str(i)) for i in range(1,31)]
days += [('may', str(i)) for i in range(1,32)]
days += [('june', str(i)) for i in range(1,31)]
days += [('july', str(i)) for i in range(1,32)]
days += [('august', str(i)) for i in range(1,32)]
days += [('september', str(i)) for i in range(1,31)]
days += [('october', str(i)) for i in range(1,32)]
days += [('november', str(i)) for i in range(1,31)]
days += [('december', str(i)) for i in range(1,32)]
print(len(days))

366


In [3]:
exception_logs = []
for month, date in days:
    print(month, date)
    url1 = "https://www.onthisday.com/day/" + month + "/" + date
    url2 = "https://www.britannica.com/on-this-day/" + month.title() + "-" + date

    onthisday = pd.DataFrame(columns=["Year", "Event"])

    try:
        res = requests.get(url1)
        if res.status_code == requests.codes.ok:
            ressoup = bs4.BeautifulSoup(res.text, "lxml")
            elems = ressoup.select("li.event-list__item")
            for e in elems:
                e = e.getText().split()
                onthisday = pd.concat([onthisday, pd.DataFrame({"Year": [e[0]], "Event": [" ".join(e[1:])]})], axis=0)
    except Exception as e:
        exception_logs.append((str(e), month, date, "1"))

    try:
        res = requests.get(url2)
        if res.status_code == requests.codes.ok:
            ressoup = bs4.BeautifulSoup(res.text, "lxml")
            onthisday = pd.concat(
                [
                    onthisday,
                    pd.DataFrame(
                        {
                            "Year": [ressoup.select(".md-history-featured-date")[0].getText().strip()],
                            "Event": [ressoup.select(".md-history-featured-content")[0].getText().strip()],
                        }
                    ),
                ],
                axis=0,
            )
            elems1 = ressoup.select(".md-history-event-date")
            elems2 = ressoup.select(".md-history-event-content")
            for e1, e2 in zip(elems1, elems2):
                onthisday = pd.concat(
                    [onthisday, pd.DataFrame({"Year": [e1.getText().strip()], "Event": [e2.getText().strip()]})],
                    axis=0,
                )

            elems1 = ressoup.select(".date.text-center")
            elems2 = ressoup.select(".md-history-born-content.grid-xs")
            for e1, e2 in zip(elems1, elems2):
                onthisday = pd.concat(
                    [
                        onthisday,
                        pd.DataFrame({"Year": [e1.getText().strip()], "Event": [e2.getText().strip() + "\nwas born"]}),
                    ],
                    axis=0,
                )
    except Exception as e:
        exception_logs.append((str(e), month, date, "2"))
    clear_output(wait=True)
    print("Exceptions Occurred: ", len(exception_logs))
    onthisday = onthisday.sort_values("Year", ascending=False).reset_index(drop=True)
    onthisday.to_json("E:/Scrapped-Data/OnThisDay/" + month + date + ".json")


Exceptions Occurred:  0
